In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA


In [20]:
df = pd.read_csv(r'E:\Omdena\Berlin-Chapter\Data Collection\Task_2_WITH_PRICE_Updated_RECLAS_OTHER (1).csv',low_memory=False)

In [21]:
df.shape

(103680, 29)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103680 entries, 0 to 103679
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   UID                    103680 non-null  int64  
 1   PRODUCT_NAME           103679 non-null  object 
 2   IMAGE_URL              103057 non-null  object 
 3   PRODUCT_PRICE          103680 non-null  float64
 4   PRODUCT_LINK           103591 non-null  object 
 5   PRODUCT_INFORMATION_T  32313 non-null   object 
 6   CATEGORY               103680 non-null  object 
 7   SUB_CATEGORY           85940 non-null   object 
 8   PRICE_PER_KG/L         79028 non-null   object 
 9   PRODUCT_QUANTITY       23968 non-null   object 
 10  SATURATED_FATTY_ACIDS  41021 non-null   float64
 11  CARBOHYDRATES          42902 non-null   float64
 12  SUGAR                  41520 non-null   float64
 13  PROTEIN                42549 non-null   float64
 14  FIBER                  5608 non-null

# Feature Selection

In [55]:
df.columns

Index(['UID', 'PRODUCT_NAME', 'IMAGE_URL', 'PRODUCT_PRICE', 'PRODUCT_LINK',
       'PRODUCT_INFORMATION_T', 'CATEGORY', 'SUB_CATEGORY', 'PRICE_PER_KG/L',
       'PRODUCT_QUANTITY', 'SATURATED_FATTY_ACIDS', 'CARBOHYDRATES', 'SUGAR',
       'PROTEIN', 'FIBER', 'SALT', 'FAT', 'INGREDIENTS', 'ALLERGENS',
       'STORAGE_INFORMATION', 'PRODUCT_REVIEWS', 'STORE_LINK', 'STORE_NAME',
       'PRICE_PER_KG/L_UNIT', 'ENERGY_KJ', 'SUB_SUB_CATEGORY', 'PRODUCT_BRAND',
       'PRODUCT_INFORMATION', 'MANUFACTURER'],
      dtype='object')

In [68]:
selected_cols =  ['PRODUCT_NAME'  , 'PRODUCT_INFORMATION_T' , 'CATEGORY' , 'SUB_CATEGORY']
new_df = df[selected_cols]

In [69]:
new_df.head()

,PRODUCT_NAME,PRODUCT_INFORMATION_T,CATEGORY,SUB_CATEGORY
0,Cruz de Malta Yerba Mate Tea Seleccion Especia...,NaN,Food & Beverage,Coffee & Tea
1,Cruz de Malta Yerba Mate Tradicional 500g -Mat...,NaN,Food & Beverage,Coffee & Tea
2,Happy Breastfeeding® - BIO mild breastfeeding ...,NaN,Food & Beverage,Coffee & Tea
3,Happy Breastfeeding® - BIO mild breastfeeding ...,NaN,Food & Beverage,Coffee & Tea
4,PregniVital® - birth preparation - especially ...,NaN,Food & Beverage,Coffee & Tea


In [70]:
new_df['PRODUCT_INFORMATION_T'] = new_df['PRODUCT_INFORMATION_T'].fillna(new_df['PRODUCT_NAME'] + " " + new_df['CATEGORY'] + " " + new_df['SUB_CATEGORY'])

C:\Users\dell\AppData\Local\Temp\ipykernel_9812\3933349191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['PRODUCT_INFORMATION_T'] = new_df['PRODUCT_INFORMATION_T'].fillna(new_df['PRODUCT_NAME'] + " " + new_df['CATEGORY'] + " " + new_df['SUB_CATEGORY'])


# Building the Recommendation Engine

In [102]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from IPython.display import Image, display, HTML

# Download NLTK resources (only need to run once)
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [79]:
def preprocess_text(text):
    if pd.isnull(text):  # Check if text is null
        return ''
    text = re.sub(r"[^a-zA-Z0-9]", " ", str(text).lower())  # Convert to string before applying lower() function
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    processed_text = " ".join(filtered_tokens)
    return processed_text

In [80]:
new_df.columns

Index(['PRODUCT_NAME', 'PRODUCT_INFORMATION_T', 'CATEGORY', 'SUB_CATEGORY'], dtype='object')

In [81]:
new_df['consolidated_text'] = new_df['PRODUCT_NAME'].apply(preprocess_text) + ' ' + new_df['PRODUCT_INFORMATION_T'].apply(preprocess_text) + ' ' + new_df['CATEGORY'].apply(preprocess_text) +' ' + new_df['SUB_CATEGORY'].apply(preprocess_text)

C:\Users\dell\AppData\Local\Temp\ipykernel_9812\1722157832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['consolidated_text'] = new_df['PRODUCT_NAME'].apply(preprocess_text) + ' ' + new_df['PRODUCT_INFORMATION_T'].apply(preprocess_text) + ' ' + new_df['CATEGORY'].apply(preprocess_text) +' ' + new_df['SUB_CATEGORY'].apply(preprocess_text)


In [83]:
# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer on the consolidated textual data
text_embeddings = tfidf_vectorizer.fit_transform(new_df['consolidated_text'])

In [95]:
processed_df = new_df
processed_df['IMAGE_URL'] = df['IMAGE_URL']
processed_df['PRODUCT_PRICE'] = df['PRODUCT_PRICE']
processed_df['PRODUCT_BRAND'] = df['PRODUCT_BRAND']

C:\Users\dell\AppData\Local\Temp\ipykernel_9812\2802522858.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['IMAGE_URL'] = df['IMAGE_URL']
C:\Users\dell\AppData\Local\Temp\ipykernel_9812\2802522858.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_df['PRODUCT_PRICE'] = df['PRODUCT_PRICE']
C:\Users\dell\AppData\Local\Temp\ipykernel_9812\2802522858.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [96]:
processed_df.head()

,PRODUCT_NAME,PRODUCT_INFORMATION_T,CATEGORY,SUB_CATEGORY,consolidated_text,IMAGE_URL,PRODUCT_PRICE,PRODUCT_BRAND
0,Cruz de Malta Yerba Mate Tea Seleccion Especia...,Cruz de Malta Yerba Mate Tea Seleccion Especia...,Food & Beverage,Coffee & Tea,cruz de malta yerba mate tea seleccion especia...,https://m.media-amazon.com/images/I/61jIegDXt7...,12.84,Besuche den Yerbox-Store
1,Cruz de Malta Yerba Mate Tradicional 500g -Mat...,Cruz de Malta Yerba Mate Tradicional 500g -Mat...,Food & Beverage,Coffee & Tea,cruz de malta yerba mate tradicional 500g mate...,https://m.media-amazon.com/images/I/61lh3d+cO8...,9.80,Besuche den Yerbee-Store
2,Happy Breastfeeding® - BIO mild breastfeeding ...,Happy Breastfeeding® - BIO mild breastfeeding ...,Food & Beverage,Coffee & Tea,happy breastfeeding bio mild breastfeeding tea...,https://m.media-amazon.com/images/I/81A6JiL4KX...,9.99,Besuche den PregniVital-Store
3,Happy Breastfeeding® - BIO mild breastfeeding ...,Happy Breastfeeding® - BIO mild breastfeeding ...,Food & Beverage,Coffee & Tea,happy breastfeeding bio mild breastfeeding tea...,https://m.media-amazon.com/images/I/81A6JiL4KX...,9.99,Besuche den PregniVital-Store
4,PregniVital® - birth preparation - especially ...,PregniVital® - birth preparation - especially ...,Food & Beverage,Coffee & Tea,pregnivital birth preparation especially 37th ...,https://m.media-amazon.com/images/I/81JPy79Vxy...,9.99,Besuche den PregniVital-Store


In [98]:
def search_products(query, top_n=5):
    # Preprocess the query
    query = preprocess_text(query)
    # Transform the query into an embedding using the TF-IDF vectorizer
    query_embedding = tfidf_vectorizer.transform([query])
    # Calculate the cosine similarity between the query embedding and all product embeddings
    similarity_scores = cosine_similarity(query_embedding, text_embeddings)
    # Get the indices of the top-N most similar products
    top_indices = similarity_scores.argsort()[0][-top_n:][::-1]
    # Retrieve the top-N recommended products
    recommendations = processed_df.iloc[top_indices]
    
    # Generate HTML representation of the results
    html_output = "<table>"
    
    for index, row in recommendations.iterrows():
        html_output += "<tr>"
        html_output += f"<td><img src='{row['IMAGE_URL']}' style='width:150px;height:150px;'></td>"
        html_output += "<td>"
        html_output += f"<b>Product Name:</b> {row['PRODUCT_NAME']}<br>"
        html_output += f"<b>Price:</b> {row['PRODUCT_PRICE']}<br>"
        html_output += f"<b>Brand:</b> {row['PRODUCT_BRAND']}<br>"
        html_output += "</td>"
        html_output += "</tr>"
    
    html_output += "</table>"
    
    # Display the HTML output
    display(HTML(html_output))

# Testing 

In [99]:
# Example usage
query = "dark chocolate"
search_products(query, top_n=5)

,Product Name: Alnatura organic fine dark chocolate 100gPrice: 1.59Brand: nan
,Product Name: Dark Nut Chocolate 100GPrice: 1.69Brand: nan
,Product Name: Brandt Minis wholemeal dark chocolate with dark chocolate 125gPrice: 3.32Brand: Zwieback
,Product Name: Lindt Dark Hazelnut 110GPrice: 1.59Brand: nan
,"Product Name: Lindt chocolate dark chocolate set | 847g | Set of 6 dark Lindt chocolate specialties (bars, LINDOR, pralines) from off-dry to dark | bulk pack | chocolate giftPrice: 28.86Brand: Besuche den Lindt-Store"


In [100]:
# Example usage
query = "lemonade"
search_products(query, top_n=5)

,Product Name: Monin Lemonade Mix 0.7lPrice: 9.95Brand: MONIN
,Product Name: Calypso Lemonade - Strawberry Lemonade - 1 x 473mlPrice: 3.99Brand: Calypso
,"Product Name: Mexican Cartel Lemonade, Lemonade, Lemonade, Cacti, 100ml, 0mgPrice: 20.79Brand: Arômes et liquides"
,Product Name: Calypso Lemonade - Tropical Mango Lemonade - 1 x 473mlPrice: 3.49Brand: Calypso
,Product Name: Calypso Lemonade - Ocean Blue Lemonade - 1 x 473mlPrice: 3.99Brand: Calypso


In [101]:
# Example usage
query = "beverage"
search_products(query, top_n=5)

,"Product Name: Beverage Powder Orange, Pack of 1 (1 x 1000 g)Price: 8.71Brand: Krüger"
,Product Name: Syrup without sugarPrice: 14.5Brand: Teisseire
,Product Name: Syrup without sugarPrice: 6.2Brand: Teisseire
,Product Name: Yes! Beverage syrup with orange flavor 0.5lPrice: 2.79Brand: nan
,Product Name: Monin coconut (1 x 0.7 l)Price: 11.69Brand: MONIN


In [109]:
# Example usage
query = "juice"
search_products(query, top_n=5)

,Product Name: High C Mild Breakfast 1lPrice: 2.39Brand: nan
,Product Name: High C Breakfast Juice 100% Juice 1lPrice: 2.29Brand: nan
,Product Name: Walthers apple juice direct juice natural (1 x 5 l juice box)Price: 14.65Brand: Walther's
,Product Name: Walthers apple juice direct juice natural (1 x 5 l juice box)Price: 13.08Brand: Walther's
,Product Name: vegetable juice 0Price: 1.29Brand: nan
